In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from imblearn.over_sampling import RandomOverSampler

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Work on AI data

In [ ]:
df = pd.read_csv("/content/AI Export.csv",sep=",", encoding='cp1252')

Work on FI data

In [ ]:
df2 = pd.read_csv("/content/FI Export.csv",sep=",", encoding='cp1252')
df2 = df2[df2['AC_MODEL']=='CRJ700']
dfFI = df2[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
#df = pd.read_csv("/content/drive/MyDrive/Capstone/Data/FI.csv",header=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AC_MODEL = CRJ700

In [ ]:
df = df[df['AC_MODEL']=='CRJ700']
df = df[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
df.append(dfFI, ignore_index=True)

In [ ]:
df['features'] = df['INTERRUPTION_REASON'].str.cat(df['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def identify_tokens1(df):
    comment = df['features']
    comment = re.sub(r'\d',' ',comment)
    words = nltk.word_tokenize(comment)
    new_words = [stemmer.stem(word) for word in words]
    comment = ' '.join(new_words)
    return comment
df['features_1'] = df.apply(identify_tokens1, axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df['features_1'].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=40000,stop_words='english')
testb = vectorizer.fit_transform(df['features_1'])

In [ ]:
#Check weights of tokens inside vectorizer
vect_score = np.asarray(testb.mean(axis=0)).ravel().tolist()
vect_array = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': vect_score})
vect_array.sort_values(by='weight',ascending=False,inplace=True)
vect_array.to_csv('tfidfWeights_withoutDigits_PorterStemmer.csv')

In [ ]:
#Check frequency of tokens inside vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=40000,stop_words='english').fit(df['features_1'])
vectorizer.vocabulary_

In [ ]:
from sklearn.model_selection import train_test_split
X = df['features_1']
y = df['ATA_CAUSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
testb.shape

(25696, 17800)

In [ ]:
testb[10,:]

<1x15816 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [ ]:
ros = RandomOverSampler()
X_over_sampled, Y_over_sampled = ros.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_over_sampled.shape

(1862112, 17800)

In [ ]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(LinearSVC())
clf.fit(X_over_sampled, Y_over_sampled)

Pipeline(memory=None,
         steps=[('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.0001, verbose=0))],
         verbose=False)

In [ ]:
clfpredicted = clf.predict(X_test)
print(metrics.accuracy_score(y_test, clfpredicted))

# New Section

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics

pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=40000,stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
# pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('clf', LogisticRegression())])
# pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('clf', KNeighborsClassifier())])
# pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('clf', DecisionTreeClassifier())])
# pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('clf', SVC())])
pipe.fit(X_train,y_train)
predicted = pipe.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


0.6178988326848249


In [ ]:
predicted[0]

261701

In [ ]:
#probs = pipe.predict_proba(X_test)
#best_n = np.argsort(-probs, axis = 1)[3:] 
b = pd.DataFrame(pipe.predict_proba([identify_tokens1({'features':'# 2 GEN Found Oil TEMP Bulb wire off'})]), columns=pipe.classes_)
#b = np.argsort(pipe.classes_, pipe.predict_proba([identify_tokens1({'features':'# 2 GEN Found Oil TEMP Bulb wire off'})])[0])
b.sort_values(by=0,axis=1,ascending=False).iloc[:, 0:3]

,7931,2411,200
0,0.300074,0.072748,0.060404


In [ ]:
pipe.classes_

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('LR', LogisticRegression())])))
pipelines.append(('ScaledLDA', Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('LDA', KNeighborsClassifier())])))
pipelines.append(('ScaledKNN', Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('KNN', LogisticRegression())])))
pipelines.append(('ScaledCART', Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('CART', DecisionTreeClassifier())])))
pipelines.append(('ScaledNB', Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('NB', GaussianNB())])))
pipelines.append(('ScaledSVM', Pipeline([('tfidf',TfidfVectorizer(max_features=4000,stop_words='english')),('SVM', SVC())])))

results = []
names=[]
for name, model in pipelines:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

Use for creating deployment model : 

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
import pickle

# AI data
df = pd.read_csv("/content/AI Export.csv",sep=",", encoding='cp1252')
df = df[df['AC_MODEL']=='CRJ']
df = df[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
# FI data (Considering AI and FI both have same preprocessing of data)
df2 = pd.read_csv("/content/FI Export.csv",sep=",", encoding='cp1252')
df2 = df2[df2['AC_MODEL']=='CRJ']
dfFI = df2[['ATA_SYMPTOM', 'ATA_CAUSE', 'INTERRUPTION_REASON', 'CORRECTIVE_ACTION']]
# Append AI and FI data
df.append(dfFI, ignore_index=True)

df['features'] = df['INTERRUPTION_REASON'].str.cat(df['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')
stemmer = PorterStemmer()
def identify_tokens1(df):
    comment = df['features']
    comment = re.sub(r'\d',' ',comment)
    words = nltk.word_tokenize(comment)
    new_words = [stemmer.stem(word) for word in words]
    comment = ' '.join(new_words)
    return comment
df['features_1'] = df.apply(identify_tokens1, axis=1)
X = df['features_1']
y = df['ATA_CAUSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe = Pipeline([('tfidf',TfidfVectorizer(max_features=40000,stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
pipe.fit(X_train,y_train)

Pkl_Filename = "linearsvc_model_crj.pkl"
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(pipe, file)
print('done')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


done
